<a href="https://colab.research.google.com/github/ktichola/Image-matching/blob/main/Matching_Challenge_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'image-matching-challenge-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F71885%2F8143495%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073113Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db2998dc19c6fea3724d4be6d9232bcfdddd44999cd765d25f2e615edf7d4caeeef74a953468334cfe3e47d0b047e91b4c25c0f5fc417911fde47f439ce54efeebe37c33fe5e4e6894506292423b9d9ce24f953ddf0b678f26b16ea8545d9907c87913a86d01f9423185399dcad35c97b4b119ffd9bec8bcd22130930fc612360680fa7e3b57890b8c7fc75db1a1dfaad0397b17f23216bb57cf90987c4c592430d5198606740885366394af916628fe7339906dbd22f4adef7797fd0655db6c920b1deadaf528a6aeebc70bc34b4cc55d0776d4ff5d17024d130781d048d535ff61a8d763e0d5f2ffec259d12acad398fac46dbfb3c5d75746ab57ff822dd796,super-glue-pretrained-network:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2058261%2F3414836%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073113Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da77b428b2ca951b83717dcd4bddbc4bd53c944e1234fa6f2dcfca24f08bb7142659154488fd804dd0ac115f79033173cbca9fe0634ca94906408f0998b2b778c87022ae5557673528bfab40c640cd466d4d861f51bb92815a3d79773523988672abab2f8e88fc81fad41e8832a9c9a44c68c1308b16843579bf3e13eaed3b3e70a88b90af0f14ed6e81db02c2df4d32f5f336710a6f69156b67c3e26b67a34122d4872bf91e68021dcabe5a402ce34004876ffd868edd0ab8fb206aec3ba4534383c360ec1c9fef874df42867355706de7d329482e1848c77e6f701e378349d5c439192b290b56678699cddca198a89dc881ddf8120d9a42d67e6a0cae6a4c94,kornia-local-feature-weights:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3117886%2F5373920%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073113Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0d55371cc9c16f96568c2947511ea01605c47e898a548e402548645a30e2db7887c5f975104b9c03ce3c9f303edb65e0a72dceb5a5e553662dae08158c2fea41a827f47f85b8efca891ccf2dc27d0cedb930e3b1a634545fd455f7261404bf19b6b88da918990bf86a789531c6494cc621159fa66d974e03cf951205b95cd66db5085c4ed9a45976fae8050226ef3f0292c8d80b1df4b73ca3875b5ca69ece879b4845847afed42572c75d1c9c39cd966543eb7791ffdc60be1ffebfa74262b6529e90d541065d76c0126afd523b185e675a0c02cdf16e233f942cdaf667592c1e253c6f2c281534e8a864e16695adc9e3aeb80a581fa6bbbb79b15e379e6b42,imc2024-packages-lightglue-rerun-kornia:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4628051%2F7884485%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073113Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D479c163e47b18f08c212ad8f434d72bf0003711e1dc6a98edd6942f3b0e4b63fca9698df7e20e49f789dce4c9f7308ac1c6c1ac5d772c279472a7dfe9876e4db35b826afe72a204f44acf0cf2d0893038348659929133c999cdde920013919c9e53ddb9ff58bb710da7ec886556b3150e5b8f4322993ce41accb8d00e8d44a61f373c0e9ab13747cab709c9af656725889947e6873d570eb83068ac86e67b48b3397bf83e97479817924435b5efed2f086e07ecb7bc354811a8259b457fbc1596ad7208448a1ec3649fe24733b266e27e7eadcffa1b5c65780a7f0bb58c194ea2044c94a58f643160ed38d606d3edfbf23f6aa0e64024187698014bd665ff3e7,tf-efficientnet/pytorch/tf-efficientnet-b7/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F2663%2F3736%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073114Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2be02940ff7a522f151fde1c0513e740242f49dfb893499fca961bfa3b1c44f7059c86c1d262e355c4a2c47543e2f41c69c19921d7880512c4a8d3b074e1299a2a1562f1dc40c4fb9a1b0c5ccbd6d5a14e9371641aa34a7224baab061dcd758f75ef58be0a29dcd1a22d6d3b54b936aeba256efcdd3d1ac6e404477321023cb4994264c342a1f722ad399ea13e98e11c5850f2acef253ce97a549e3fca3d82e3535724901eedb8d694dd39cf28f89c3cb56cf85d95f9be716c1283d155f8301161623b82a39c0bb1cc87c951fcaa4997c060ca119e36fc4038c4af8871acc81d8b48a4d1a08fb30dc07ba3cf80e6ff370fce703f23b7f52b66aaa9b6378cae27,loftr/pytorch/outdoor/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F2742%2F3840%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073114Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dabef4968cce2b3e643b4a114ad469f1cead778e24520bda42122ff200363a503c4f2a8293c7ac84b9e0a704d4f87e89367858010da297cc87c6e22a17b28a19d168b41edf9c3fbc9fb9a0269dbacb6cb51ae37aa5e2f2c3385cfcf2660b4773bda0d68d2b3d8a75109d78421770b584ade791d1eb908f3a40e70f6840d46d4bce6d58b26b6b15029bd0fd07243a0ecdd5962510678025bd3306a6387336d722c9f4db385f5e0d72b82a42e79276a3f77e1bb0586a7b8bb9fd7d9a53446c73b867cb8e4e6f55d32c424d16c67059e04982017a323c657919bf40b1610d207f5d5203f1c6916eb5d797d37b2a87dd9d96e99fa05f757883f2ccdf4eab13919b713,disk/pytorch/depth-supervision/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F2747%2F3846%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073114Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6d1f498bfc92aaa6156707d831e329e3277523acfe66af0d0d3f2f4313dba8c150ea18f7e65d45b6e4ca81d7c9c49fda05d9590f11006bfa677769018c5bac8ee26f75ccfb4ee8c8b761449c2f570588e41f313f252c06f42d288944aeb18f8ffc4159895f9f76389338c45a150e8d19a60c8f70c5912be9f96c5779c68d21485a408f38fa3ceba9fbe24db5e89ab4e8b93e4c6c0e8b5a40eca388b00ae600914cc44e2ec40a6b66fa2fb605e9dc0c92caeaa503dcdc2b1a7b7d3bca476ecb58841d53ea522890825c90fb0ba42b26b726ec573812f2d332a03319046490d057541e1b6a18ebd3854f3b1ebaa5402bb6241eccd5c5bad490bab98beb905e9fc4,dinov2/pytorch/base/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F3326%2F4534%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073114Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D64736f45b303be6ad0efebeb7a2000d1cc975cbdce81226a167c05e9a9e6de4e8aebc7d388a881fc6023e0906cb183575b1f37260bd7353e1aaa3ce8f5dbd6f2111e339a9e14fee154885536349fb0150e5812775d2e848f6b4d137842412c82910a7cff2aee03d84db51335c5085a59b378076287d7377d1b9a893de602400e8c5d818258b2aad0a6078067eb7302226944fccc619d59935c4ea172da78d10ad4e5f9759239ddbda439498f649d4f69806a6d29fe546e1c20b1096449bc8381f82b9039122f8995952163392905f71dc1838b8778a92ed690630ccad62f0fb2b83c71e945b3b70bb88f4d3bfc4681278e0544caea1c0483c6f840e8e0d5936f,lightglue/pytorch/aliked/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F14317%2F17191%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073114Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9dfedc83b84d7ab5c3f6fe6852891be9f460890527b74984ee0fbb5a6f0f0afa4d7081b9f16c7ccc843558ab8aead81c0a0571e55b5fcda4c933708bd676b552680b2ddc73a946df6354313f3c3095d4ac9b470e824d039eeca56b93850f989b1625bf98af7f334db362167819eefde8b82ba07c4105fc108b044af034a4a87c527074e470a7aba9d5d07ef98ef0ebe5cfda41cfdb4042a509673aa79d80e94e239204bf81c26a15150d3430402a5273c763ffe3593c32a0450df9edfb6f220421272cad32ac4b71f84c1e1eaff162b19fb71090b4641ffe7e6b979a02985b3963f8d347c4c6a189ecd797468969c445d20c9f0c6ad527d524aaec3c5b7db4e5,aliked/pytorch/aliked-n16/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F14611%2F17555%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073114Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8f0e9d925f046ba96e8b240b9114134092b520a6de5f17aef04508b00462411fd5e27e40188d7f2151d3353d7fdcfdc256157a70d17b84d9a91c357b9fec88c35d6e34a97382b297f508ac3412766988019483afaf0849517259b265746c1948598f21b50e54041addcd47efbfe6d4cda1767f601105c86d34da141c8f3e0765cd85b71e2e961fb19a740c8d21a43f254d8061b0330b51c83be2ff4f1a5203f76467e4ee4a3fda605b60aee5f540c38732cc0055b4591ced5578ce4f75c484b1ce66517447cfa3b460f4a21e6b1e34c34cc46e186338ecb0e8d6ba5df8647370aef89fba17f252ed5f4245864786ee98dd59bea61a48ac62bff62b9d4c39c72a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Dependencies

In [ ]:
!python -m pip install --no-deps /kaggle/input/dependencies-imc/pycolmap/pycolmap-0.4.0-cp310-cp310-manylinux2014_x86_64.whl
!python -m pip install --no-deps /kaggle/input/dependencies-imc/safetensors/safetensors-0.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!python -m pip install --no-index --find-links=/kaggle/input/dependencies-imc/transformers/ transformers > /dev/null
!python -m pip install  --no-deps /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl

# dkm
!python -m pip install --no-index --find-links=/kaggle/input/dkm-dependencies/packages einops > /dev/null

Processing /kaggle/input/dependencies-imc/pycolmap/pycolmap-0.4.0-cp310-cp310-manylinux2014_x86_64.whl
Processing /kaggle/input/dependencies-imc/safetensors/safetensors-0.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.4.3
    Uninstalling safetensors-0.4.3:
      Successfully uninstalled safetensors-0.4.3
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl


In [ ]:
# lightglue models
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/* /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/* /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv-pth
!cp /kaggle/input/pytorch-lightglue-models/* /root/.cache/torch/hub/checkpoints/

# dkm model
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/dkm-dependencies/DKMv3_outdoor.pth /root/.cache/torch/hub/checkpoints/

In [ ]:
%matplotlib inline

In [ ]:
# General utilities
import os
from tqdm import tqdm
from time import time
from fastprogress import progress_bar
import gc
import numpy as np
import pandas as pd
import h5py
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy

# CV/ML
import cv2
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import kornia as K
import kornia.feature as KF
from PIL import Image
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

import torchvision

# 3D reconstruction
import pycolmap

import glob
import matplotlib
from matplotlib import pyplot as plt

# dkm
import sys
sys.path.append('/kaggle/input/dkm-dependencies/DKM/')
from dkm.utils.utils import tensor_to_pil, get_tuple_transform_ops
from dkm import DKMv3_outdoor

# LoFTR
from kornia.feature import LoFTR

In [ ]:
from lightglue import match_pair
from lightglue import ALIKED, SuperPoint, DoGHardNet, LightGlue
from lightglue.utils import load_image, rbd
from kornia.feature import LoFTR

In [ ]:
print('Kornia version', K.__version__)
print('Pycolmap version', pycolmap.__version__)

Kornia version 0.7.2
Pycolmap version 0.4.0


# Configurations

In [ ]:
class CONFIG:
    use_aliked_lightglue = True
    use_doghardnet_lightglue = False
    use_superpoint_lightglue = False
    use_loftr = False
    use_dkm = False
    use_superglue = False

    params_aliked_lightglue = {
        "num_features" : 8192,
        "detection_threshold" : 0.001,
        "min_matches" : 15,
        "resize_to" : 1024,
    }

    params_doghardnet_lightglue = {
        "num_features" : 8192,
        "detection_threshold" : 0.001,
        "min_matches" : 15,
        "resize_to" : 1024,
    }

    params_superpoint_lightglue = {
        "num_features" : 4096,
        "detection_threshold" : 0.005,
        "min_matches" : 15,
        "resize_to" : 1024,
    }

    params_loftr = {
        "resize_small_edge_to" : 750,
        "min_matches" : 15,
    }

    params_dkm = {
        "num_features" : 2048,
        "detection_threshold" : 0.4,
        "min_matches" : 15,
        "resize_to" : (540, 720),
    }

    params_sg = {
        "sg_config" :
        {
            "superpoint": {
                "nms_radius": 4,
                "keypoint_threshold": 0.0001,
                "max_keypoints": 4096
            },
            "superglue": {
                "weights": "outdoor",
                "sinkhorn_iterations": 10,
                "match_threshold": 0.2,
            },
        },
        "resize_to": 1240,
        "min_matches": 15,
    }

In [ ]:
device=torch.device('cuda')

# COLMAP utilities

In [ ]:
# Code to manipulate a colmap database.
# Forked from https://github.com/colmap/colmap/blob/dev/scripts/python/database.py

# Copyright (c) 2018, ETH Zurich and UNC Chapel Hill.
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
#     * Redistributions of source code must retain the above copyright
#       notice, this list of conditions and the following disclaimer.
#
#     * Redistributions in binary form must reproduce the above copyright
#       notice, this list of conditions and the following disclaimer in the
#       documentation and/or other materials provided with the distribution.
#
#     * Neither the name of ETH Zurich and UNC Chapel Hill nor the names of
#       its contributors may be used to endorse or promote products derived
#       from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
# ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDERS OR CONTRIBUTORS BE
# LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR
# CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF
# SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS
# INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN
# CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
# ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.
#
# Author: Johannes L. Schoenberger (jsch-at-demuc-dot-de)

# This script is based on an original implementation by True Price.

import sys
import sqlite3
import numpy as np


IS_PYTHON3 = sys.version_info[0] >= 3

MAX_IMAGE_ID = 2**31 - 1

CREATE_CAMERAS_TABLE = """CREATE TABLE IF NOT EXISTS cameras (
    camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    model INTEGER NOT NULL,
    width INTEGER NOT NULL,
    height INTEGER NOT NULL,
    params BLOB,
    prior_focal_length INTEGER NOT NULL)"""

CREATE_DESCRIPTORS_TABLE = """CREATE TABLE IF NOT EXISTS descriptors (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)"""

CREATE_IMAGES_TABLE = """CREATE TABLE IF NOT EXISTS images (
    image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL UNIQUE,
    camera_id INTEGER NOT NULL,
    prior_qw REAL,
    prior_qx REAL,
    prior_qy REAL,
    prior_qz REAL,
    prior_tx REAL,
    prior_ty REAL,
    prior_tz REAL,
    CONSTRAINT image_id_check CHECK(image_id >= 0 and image_id < {}),
    FOREIGN KEY(camera_id) REFERENCES cameras(camera_id))
""".format(MAX_IMAGE_ID)

CREATE_TWO_VIEW_GEOMETRIES_TABLE = """
CREATE TABLE IF NOT EXISTS two_view_geometries (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    config INTEGER NOT NULL,
    F BLOB,
    E BLOB,
    H BLOB)
"""

CREATE_KEYPOINTS_TABLE = """CREATE TABLE IF NOT EXISTS keypoints (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)
"""

CREATE_MATCHES_TABLE = """CREATE TABLE IF NOT EXISTS matches (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB)"""

CREATE_NAME_INDEX = \
    "CREATE UNIQUE INDEX IF NOT EXISTS index_name ON images(name)"

CREATE_ALL = "; ".join([
    CREATE_CAMERAS_TABLE,
    CREATE_IMAGES_TABLE,
    CREATE_KEYPOINTS_TABLE,
    CREATE_DESCRIPTORS_TABLE,
    CREATE_MATCHES_TABLE,
    CREATE_TWO_VIEW_GEOMETRIES_TABLE,
    CREATE_NAME_INDEX
])


def image_ids_to_pair_id(image_id1, image_id2):
    if image_id1 > image_id2:
        image_id1, image_id2 = image_id2, image_id1
    return image_id1 * MAX_IMAGE_ID + image_id2


def pair_id_to_image_ids(pair_id):
    image_id2 = pair_id % MAX_IMAGE_ID
    image_id1 = (pair_id - image_id2) / MAX_IMAGE_ID
    return image_id1, image_id2


def array_to_blob(array):
    if IS_PYTHON3:
        return array.tostring()
    else:
        return np.getbuffer(array)


def blob_to_array(blob, dtype, shape=(-1,)):
    if IS_PYTHON3:
        return np.fromstring(blob, dtype=dtype).reshape(*shape)
    else:
        return np.frombuffer(blob, dtype=dtype).reshape(*shape)


class COLMAPDatabase(sqlite3.Connection):

    @staticmethod
    def connect(database_path):
        return sqlite3.connect(database_path, factory=COLMAPDatabase)


    def __init__(self, *args, **kwargs):
        super(COLMAPDatabase, self).__init__(*args, **kwargs)

        self.create_tables = lambda: self.executescript(CREATE_ALL)
        self.create_cameras_table = \
            lambda: self.executescript(CREATE_CAMERAS_TABLE)
        self.create_descriptors_table = \
            lambda: self.executescript(CREATE_DESCRIPTORS_TABLE)
        self.create_images_table = \
            lambda: self.executescript(CREATE_IMAGES_TABLE)
        self.create_two_view_geometries_table = \
            lambda: self.executescript(CREATE_TWO_VIEW_GEOMETRIES_TABLE)
        self.create_keypoints_table = \
            lambda: self.executescript(CREATE_KEYPOINTS_TABLE)
        self.create_matches_table = \
            lambda: self.executescript(CREATE_MATCHES_TABLE)
        self.create_name_index = lambda: self.executescript(CREATE_NAME_INDEX)

    def add_camera(self, model, width, height, params,
                   prior_focal_length=False, camera_id=None):
        params = np.asarray(params, np.float64)
        cursor = self.execute(
            "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
            (camera_id, model, width, height, array_to_blob(params),
             prior_focal_length))
        return cursor.lastrowid

    def add_image(self, name, camera_id,
                  prior_q=np.zeros(4), prior_t=np.zeros(3), image_id=None):
        cursor = self.execute(
            "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (image_id, name, camera_id, prior_q[0], prior_q[1], prior_q[2],
             prior_q[3], prior_t[0], prior_t[1], prior_t[2]))
        return cursor.lastrowid

    def add_keypoints(self, image_id, keypoints):
        assert(len(keypoints.shape) == 2)
        assert(keypoints.shape[1] in [2, 4, 6])

        keypoints = np.asarray(keypoints, np.float32)
        self.execute(
            "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
            (image_id,) + keypoints.shape + (array_to_blob(keypoints),))

    def add_descriptors(self, image_id, descriptors):
        descriptors = np.ascontiguousarray(descriptors, np.uint8)
        self.execute(
            "INSERT INTO descriptors VALUES (?, ?, ?, ?)",
            (image_id,) + descriptors.shape + (array_to_blob(descriptors),))

    def add_matches(self, image_id1, image_id2, matches):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        self.execute(
            "INSERT INTO matches VALUES (?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches),))

    def add_two_view_geometry(self, image_id1, image_id2, matches,
                              F=np.eye(3), E=np.eye(3), H=np.eye(3), config=2):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        F = np.asarray(F, dtype=np.float64)
        E = np.asarray(E, dtype=np.float64)
        H = np.asarray(H, dtype=np.float64)
        self.execute(
            "INSERT INTO two_view_geometries VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches), config,
             array_to_blob(F), array_to_blob(E), array_to_blob(H)))

# h5 to colmap db

In [ ]:
# Code to interface DISK with Colmap.
# Forked from https://github.com/cvlab-epfl/disk/blob/37f1f7e971cea3055bb5ccfc4cf28bfd643fa339/colmap/h5_to_db.py

#  Copyright [2020] [Michał Tyszkiewicz, Pascal Fua, Eduard Trulls]
#
#   Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at
#
#       http://www.apache.org/licenses/LICENSE-2.0
#
#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.

import os, argparse, h5py, warnings
import numpy as np
from tqdm import tqdm
from PIL import Image, ExifTags


def get_focal(image_path, err_on_default=False):
    image         = Image.open(image_path)
    max_size      = max(image.size)

    exif = image.getexif()
    focal = None
    if exif is not None:
        focal_35mm = None
        # https://github.com/colmap/colmap/blob/d3a29e203ab69e91eda938d6e56e1c7339d62a99/src/util/bitmap.cc#L299
        for tag, value in exif.items():
            focal_35mm = None
            if ExifTags.TAGS.get(tag, None) == 'FocalLengthIn35mmFilm':
                focal_35mm = float(value)
                break

        if focal_35mm is not None:
            focal = focal_35mm / 35. * max_size

    if focal is None:
        if err_on_default:
            raise RuntimeError("Failed to find focal length")

        # failed to find it in exif, use prior
        FOCAL_PRIOR = 1.2
        focal = FOCAL_PRIOR * max_size

    return focal

def create_camera(db, image_path, camera_model):
    image         = Image.open(image_path)
    width, height = image.size

    focal = get_focal(image_path)

    if camera_model == 'simple-pinhole':
        model = 0 # simple pinhole
        param_arr = np.array([focal, width / 2, height / 2])
    if camera_model == 'pinhole':
        model = 1 # pinhole
        param_arr = np.array([focal, focal, width / 2, height / 2])
    elif camera_model == 'simple-radial':
        model = 2 # simple radial
        param_arr = np.array([focal, width / 2, height / 2, 0.1])
    elif camera_model == 'opencv':
        model = 4 # opencv
        param_arr = np.array([focal, focal, width / 2, height / 2, 0., 0., 0., 0.])

    return db.add_camera(model, width, height, param_arr)


def add_keypoints(db, h5_path, image_path, img_ext, camera_model, single_camera = True):
    keypoint_f = h5py.File(os.path.join(h5_path, 'keypoints.h5'), 'r')

    camera_id = None
    fname_to_id = {}
    for filename in tqdm(list(keypoint_f.keys())):
        keypoints = keypoint_f[filename][()]

        fname_with_ext = filename# + img_ext
        path = os.path.join(image_path, fname_with_ext)
        if not os.path.isfile(path):
            raise IOError(f'Invalid image path {path}')

        if camera_id is None or not single_camera:
            camera_id = create_camera(db, path, camera_model)
        image_id = db.add_image(fname_with_ext, camera_id)
        fname_to_id[filename] = image_id

        db.add_keypoints(image_id, keypoints)

    return fname_to_id

def add_matches(db, h5_path, fname_to_id):
    match_file = h5py.File(os.path.join(h5_path, 'matches.h5'), 'r')

    added = set()
    n_keys = len(match_file.keys())
    n_total = (n_keys * (n_keys - 1)) // 2

    with tqdm(total=n_total) as pbar:
        for key_1 in match_file.keys():
            group = match_file[key_1]
            for key_2 in group.keys():
                id_1 = fname_to_id[key_1]
                id_2 = fname_to_id[key_2]

                pair_id = image_ids_to_pair_id(id_1, id_2)
                if pair_id in added:
                    warnings.warn(f'Pair {pair_id} ({id_1}, {id_2}) already added!')
                    continue

                matches = group[key_2][()]
                db.add_matches(id_1, id_2, matches)

                added.add(pair_id)

                pbar.update(1)

def import_into_colmap(img_dir,
                       feature_dir ='.featureout',
                       database_path = 'colmap.db',
                       img_ext='.jpg'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, img_ext, 'simple-radial', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )

    db.commit()
    return

# Image Pairs

In [ ]:
# We will use ViT global descriptor to get matching shortlists.
def get_global_desc(fnames, model,
                    device =  torch.device('cpu')):
    model = model.eval()
    model= model.to(device)
    config = resolve_data_config({}, model=model)
    transform = create_transform(**config)
    global_descs_convnext=[]
    for i, img_fname_full in tqdm(enumerate(fnames),total= len(fnames)):
        key = os.path.splitext(os.path.basename(img_fname_full))[0]
        img = Image.open(img_fname_full).convert('RGB')
        timg = transform(img).unsqueeze(0).to(device)
        with torch.no_grad(), torch.cuda.amp.autocast():
            desc = model.forward_features(timg.to(device)).mean(dim=(-1,2))#
            #print (desc.shape)
            desc = desc.view(1, -1)
            desc_norm = F.normalize(desc, dim=1, p=2)
        #print (desc_norm)
        global_descs_convnext.append(desc_norm.detach().cpu())
    global_descs_all = torch.cat(global_descs_convnext, dim=0)
    return global_descs_all.to(torch.float32)

def convert_1d_to_2d(idx, num_images):
    idx1 = idx // num_images
    idx2 = idx % num_images
    return (idx1, idx2)

def get_pairs_from_distancematrix(mat):
    pairs = [ convert_1d_to_2d(idx, mat.shape[0]) for idx in np.argsort(mat.flatten())]
    pairs = [ pair for pair in pairs if pair[0] < pair[1] ]
    return pairs

def get_img_pairs_exhaustive(img_fnames, model, device):
    #index_pairs = []
    #for i in range(len(img_fnames)):
    #    for j in range(i+1, len(img_fnames)):
    #        index_pairs.append((i,j))
    #return index_pairs
    descs = get_global_desc(img_fnames, model, device=device)
    dm = torch.cdist(descs, descs, p=2).detach().cpu().numpy()
    matching_list = get_pairs_from_distancematrix(dm)
    return matching_list


def get_image_pairs_shortlist(fnames,
                              sim_th = 0.6, # should be strict
                              min_pairs = 20,
                              exhaustive_if_less = 20,
                              device=torch.device('cpu')):
    num_imgs = len(fnames)

    model = timm.create_model('tf_efficientnet_b7',
                              checkpoint_path='/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b7/1/tf_efficientnet_b7_ra-6c08e654.pth')
    model.eval()
    descs = get_global_desc(fnames, model, device=device)

    if num_imgs <= exhaustive_if_less:
        return get_img_pairs_exhaustive(fnames, model, device)

    dm = torch.cdist(descs, descs, p=2).detach().cpu().numpy()
    # removing half
    mask = dm <= sim_th
    total = 0
    matching_list = []
    ar = np.arange(num_imgs)
    already_there_set = []
    for st_idx in range(num_imgs-1):
        mask_idx = mask[st_idx]
        to_match = ar[mask_idx]
        if len(to_match) < min_pairs:
            to_match = np.argsort(dm[st_idx])[:min_pairs]
        for idx in to_match:
            if st_idx == idx:
                continue
            if dm[st_idx, idx] < 1000:
                matching_list.append(tuple(sorted((st_idx, idx.item()))))
                total+=1
    matching_list = sorted(list(set(matching_list)))
    return matching_list

# Keypoints: LightGlue series

In [ ]:
def load_torch_image(fname, device=torch.device('cpu')):
    img = K.io.load_image(fname, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img

def detect_common(img_fnames,
                  model_name,
                  feature_dir = '.featureout',
                  num_features = 4096,
                  resize_to = 1024,
                  detection_threshold = 0.01,
                  device=torch.device('cpu')):

    dict_model = {
        "aliked" : ALIKED,
        "superpoint" : SuperPoint,
        "doghardnet" : DoGHardNet,
    }
    extractor_class = dict_model[model_name]

    dtype = torch.float32 # ALIKED has issues with float16
    extractor = extractor_class(
        max_num_keypoints=num_features, detection_threshold=detection_threshold, resize=resize_to
    ).eval().to(device, dtype)
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)
    with h5py.File(f'{feature_dir}/keypoints_{model_name}.h5', mode='w') as f_kp, \
         h5py.File(f'{feature_dir}/descriptors_{model_name}.h5', mode='w') as f_desc:
        for img_path in tqdm(img_fnames):
            img_fname = img_path.split('/')[-1]
            key = img_fname
            with torch.inference_mode():
                image0 = load_torch_image(img_path, device=device).to(dtype)
                feats0 = extractor.extract(image0)  # auto-resize the image, disable with resize=None
                kpts = feats0['keypoints'].reshape(-1, 2).detach().cpu().numpy()
                descs = feats0['descriptors'].reshape(len(kpts), -1).detach().cpu().numpy()
                f_kp[key] = kpts
                f_desc[key] = descs
                print(f"{model_name} > kpts.shape={kpts.shape}, descs.shape={descs.shape}")
    return

def match_with_lightglue_common(img_fnames, model_name,
                   index_pairs, file_keypoints,
                   feature_dir = '.featureout',
                   device=torch.device('cpu'),
                   min_matches=15,verbose=True):
    lg_matcher = KF.LightGlueMatcher(model_name, {"width_confidence": -1,
                                                "depth_confidence": -1,
                                                 "mp": True if 'cuda' in str(device) else False}).eval().to(device)

    cnt_pairs = 0
    with h5py.File(f'{feature_dir}/keypoints_{model_name}.h5', mode='r') as f_kp, \
        h5py.File(f'{feature_dir}/descriptors_{model_name}.h5', mode='r') as f_desc, \
        h5py.File(file_keypoints, mode='w') as f_match:
        for pair_idx in tqdm(index_pairs):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]

            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
            kp1 = torch.from_numpy(f_kp[key1][...]).to(device)
            kp2 = torch.from_numpy(f_kp[key2][...]).to(device)
            desc1 = torch.from_numpy(f_desc[key1][...]).to(device)
            desc2 = torch.from_numpy(f_desc[key2][...]).to(device)
            with torch.inference_mode():
                dists, idxs = lg_matcher(desc1,
                                         desc2,
                                         KF.laf_from_center_scale_ori(kp1[None]),
                                         KF.laf_from_center_scale_ori(kp2[None]))
            if len(idxs)  == 0:
                continue
            n_matches = len(idxs)
            kp1 = kp1[idxs[:,0], :].cpu().numpy().reshape(-1, 2).astype(np.float32)
            kp2 = kp2[idxs[:,1], :].cpu().numpy().reshape(-1, 2).astype(np.float32)
            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                group.create_dataset(key2, data=np.concatenate([kp1, kp2], axis=1))
                cnt_pairs+=1
                print (f'{key1}-{key2}: {n_matches} matches @ {cnt_pairs}th pair({model_name}+lightglue)')
            else:
                print (f'{key1}-{key2}: {n_matches} matches --> skipped')
    return

def detect_lightglue_common(
    img_fnames, model_name, index_pairs, feature_dir, device, file_keypoints,
    resize_to=1024,
    detection_threshold=0.01,
    num_features=4096,
    min_matches=15,
):
    t=time()
    detect_common(
        img_fnames, model_name, feature_dir,
        resize_to=resize_to,
        num_features=num_features,
        detection_threshold=detection_threshold,
        device=device
    )
    gc.collect()
    match_with_lightglue_common(
        img_fnames, model_name, index_pairs, file_keypoints,
        feature_dir=feature_dir,
        min_matches=min_matches,
        device=device
    )
    t=time() -t
    print(f'Features matched in  {t:.4f} sec ({model_name}+LightGlue)')
    return t

# Keypoints: SuperGlue

In [ ]:
import sys
sys.path.append("../input/super-glue-pretrained-network")
from models.matching import Matching
from models.utils import (compute_pose_error, compute_epipolar_error,
                          estimate_pose, make_matching_plot,
                          error_colormap, AverageTimer, pose_auc, read_image,
                          process_resize, frame2tensor,
                          rotate_intrinsics, rotate_pose_inplane,
                          scale_intrinsics)

from torch.nn import functional as torchF  # For resizing tensor

def sg_imread(path):
    image = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    return image

# Preprocess
def sg_read_image(image, device, resize):
    w, h = image.shape[1], image.shape[0]
    w_new, h_new = process_resize(w, h, [resize,])

    unit_shape = 8
    w_new = w_new // unit_shape * unit_shape
    h_new = h_new // unit_shape * unit_shape

    scales = (float(w) / float(w_new), float(h) / float(h_new))
    image = cv2.resize(image.astype('float32'), (w_new, h_new))

    inp = frame2tensor(image, "cpu")
    return image, inp, scales, (h, w)

class SGDataset(Dataset):
    def __init__(self, fnames1, fnames2, resize_to, device):
        self.fnames1 = fnames1
        self.fnames2 = fnames2
        self.resize_to = resize_to
        self.device = device

    def __len__(self):
        return len(self.fnames1)

    def __getitem__(self, idx):
        fname1 = self.fnames1[idx]
        fname2 = self.fnames2[idx]

        im1, im2 = cv2.imread(fname1, cv2.IMREAD_GRAYSCALE), cv2.imread(fname2, cv2.IMREAD_GRAYSCALE)
        _, image1, scale1, ori_shape1 = sg_read_image(im1, self.device, self.resize_to)
        _, image2, scale2, ori_shape2 = sg_read_image(im2, self.device, self.resize_to)
        return image1, image2, torch.tensor([idx]), torch.tensor(ori_shape1), torch.tensor(ori_shape2)

def get_superglue_dataloader(images1, images2, resize_to, device, batch_size=1):
    dataset = SGDataset(images1, images2, resize_to, device)
    dataloader = DataLoader(
        dataset=dataset,
        shuffle=False,
        batch_size=batch_size,
        pin_memory=True,
        num_workers=2,
        drop_last=False
    )
    return dataloader

def detect_superglue(
    img_fnames, index_pairs, feature_dir, device, sg_config, file_keypoints,
    resize_to=750, min_matches=15
):
    t=time()

    matcher_superglue = Matching(sg_config).eval().to(device)

    fnames1, fnames2, idxs1, idxs2 = [], [], [], []
    for pair_idx in progress_bar(index_pairs):
        idx1, idx2 = pair_idx
        fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
        fnames1.append(fname1)
        fnames2.append(fname2)
        idxs1.append(idx1)
        idxs2.append(idx2)

    dataloader = get_superglue_dataloader( fnames1, fnames2, resize_to, device)

    cnt_pairs = 0

    with h5py.File(file_keypoints, mode='w') as f_match:
        for X in dataloader:
            image1, image2, idx, ori_shape_1, ori_shape_2 = X

            fname1, fname2 = fnames1[idx], fnames2[idx]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]

            pred = matcher_superglue({"image0": image1[0].to(device), "image1": image2[0].to(device)})
            pred = {k: v[0].detach().cpu().numpy().copy() for k, v in pred.items()}
            mkpts1, mkpts2 = pred["keypoints0"], pred["keypoints1"]
            matches, conf = pred["matches0"], pred["matching_scores0"]

            valid = matches > -1
            mkpts1 = mkpts1[valid]
            mkpts2 = mkpts2[matches[valid]]
            mconf = conf[valid]

            ori_shape_1 = ori_shape_1[0].numpy()
            ori_shape_2 = ori_shape_2[0].numpy()

            # Scaling coords
            mkpts1[:,0] = mkpts1[:,0] * ori_shape_1[1] / image1[0].shape[3]   # X
            mkpts1[:,1] = mkpts1[:,1] * ori_shape_1[0] / image1[0].shape[2]   # Y
            mkpts2[:,0] = mkpts2[:,0] * ori_shape_2[1] / image2[0].shape[3]   # X
            mkpts2[:,1] = mkpts2[:,1] * ori_shape_2[0] / image2[0].shape[2]   # Y

            n_matches = mconf.shape[0]

            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                group.create_dataset(key2, data=np.concatenate([mkpts1, mkpts2], axis=1).astype(np.float32))
                cnt_pairs+=1
                print (f'{key1}-{key2}: {n_matches} matches @ {cnt_pairs}th pair(superglue)')
            else:
                print (f'{key1}-{key2}: {n_matches} matches --> skipped')

    gc.collect()
    t=time() -t
    print(f'Features matched in  {t:.4f} sec')
    return t

# Keypoints: DKM

In [ ]:
class DKMDataset(Dataset):
    def __init__(self, fnames1, fnames2, resize_to, device):
        self.fnames1 = fnames1
        self.fnames2 = fnames2
        self.resize_to = resize_to
        self.device = device
        self.test_transform = get_tuple_transform_ops(
            resize=self.resize_to, normalize=True
        )


    def __len__(self):
        return len(self.fnames1)

    def __getitem__(self, idx):
        fname1 = self.fnames1[idx]
        fname2 = self.fnames2[idx]

        im1, im2 = Image.open(fname1), Image.open(fname2)
        ori_shape_1 = im1.size
        ori_shape_2 = im2.size
        image1, image2 = self.test_transform((im1, im2))
        return image1, image2, torch.tensor([idx]), torch.tensor(ori_shape_1), torch.tensor(ori_shape_2)

def get_dkm_dataloader(images1, images2, resize_to, device, batch_size=4):
    dataset = DKMDataset(images1, images2, resize_to, device)
    dataloader = DataLoader(
        dataset=dataset,
        shuffle=False,
        batch_size=batch_size,
        pin_memory=True,
        num_workers=2,
        drop_last=False
    )
    return dataloader

def get_dkm_mkpts(dkm_model, bimgs1, bimgs2, shapes1, shapes2, detection_threshold=0.5, num_features = 2000, min_matches=15):
    dense_matches, dense_certainty = dkm_model.match(bimgs1, bimgs2, batched=True)
    print("***", dense_matches.shape, dense_certainty.shape)

    store_mkpts1, store_mkpts2, store_mconf = [], [], []
    # drop low confidence pairs
    for b in range(dense_matches.shape[0]):
        u_dense_matches = dense_matches[b, dense_certainty[b,...].sqrt() >= detection_threshold, :]
        u_dense_certainty = dense_certainty[b, dense_certainty[b,...].sqrt() >= detection_threshold]

        if u_dense_matches.shape[0] > num_features:
            u_dense_matches, u_dense_certainty = dkm_model.sample( u_dense_matches, u_dense_certainty, num=num_features)

        u_dense_matches = u_dense_matches.reshape((-1, 4))
        u_dense_certainty = u_dense_certainty.reshape((-1,))

        mkpts1 = u_dense_matches[:, :2]
        mkpts2 = u_dense_matches[:, 2:]

        w1, h1 = shapes1[b, :]
        w2, h2 = shapes2[b, :]

        mkpts1[:, 0] = ((mkpts1[:, 0] + 1)/2) * w1
        mkpts1[:, 1] = ((mkpts1[:, 1] + 1)/2) * h1

        mkpts2[:, 0] = ((mkpts2[:, 0] + 1)/2) * w2
        mkpts2[:, 1] = ((mkpts2[:, 1] + 1)/2) * h2

        mkpts1 = mkpts1.cpu().detach().numpy()
        mkpts2 = mkpts2.cpu().detach().numpy()
        mconf  = u_dense_certainty.sqrt().cpu().detach().numpy()


        if mconf.shape[0] > min_matches:
            try:
                # calc Fundamental matrix from keypoints
                F, inliers = cv2.findFundamentalMat(mkpts1, mkpts2, cv2.USAC_MAGSAC, 0.200, 0.999, 2000)
                inliers = inliers > 0
                mkpts1 = mkpts1[inliers[:,0]]
                mkpts2 = mkpts2[inliers[:,0]]
                mconf  = mconf[inliers[:,0]]
                #print("---", mconf.shape)
                if mconf.shape[0] > 3000:
                    rand_idx = np.random.choice(range(mconf.shape[0]), 3000, replace=False)
                    mkpts1 = mkpts1[rand_idx, :]
                    mkpts2 = mkpts2[rand_idx, :]
                    mconf  = mconf[rand_idx]
            except:
                mkpts1 = np.empty((0,2))
                mkpts2 = np.empty((0,2))
                mconf = np.empty((0,))

        store_mkpts1.append(mkpts1)
        store_mkpts2.append(mkpts2)
        store_mconf.append(mconf)
    return store_mkpts1, store_mkpts2, store_mconf

def detect_dkm(
    img_fnames, index_pairs, feature_dir, device,
    resize_to=(540, 720),
    detection_threshold=0.4,
    num_features=2000,
    min_matches=15,
):
    t=time()
    dkm_model = DKMv3_outdoor(device=device)
    dkm_model.upsample_preds=False

    fnames1, fnames2 = [], []
    for pair_idx in progress_bar(index_pairs):
        idx1, idx2 = pair_idx
        fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
        fnames1.append(fname1)
        fnames2.append(fname2)

    cnt_pairs = 0
    with h5py.File(f'{feature_dir}/matches_dkm.h5', mode='w') as f_match:
        dataloader = get_dkm_dataloader(fnames1, fnames2, resize_to, device, batch_size=4)
        for X in tqdm(dataloader):
            images1, images2, idxs, shapes1, shapes2 = X
            store_mkpts1, store_mkpts2, store_mconf = get_dkm_mkpts(
                dkm_model, images1.to(device), images2.to(device), shapes1, shapes2,
                detection_threshold=detection_threshold, num_features = num_features, min_matches=min_matches,
            )

            for b in range(images1.shape[0]):
                mkpts1 = store_mkpts1[b]
                mkpts2 = store_mkpts2[b]
                mconf = store_mconf[b]
                file1 = fnames1[idxs[b]]
                file2 = fnames2[idxs[b]]
                key1, key2 = file1.split('/')[-1], file2.split('/')[-1]

                n_matches = mconf.shape[0]
                print (f'{key1}-{key2}: {n_matches} matches @ {cnt_pairs}th pair(dkm)')

                group  = f_match.require_group(key1)
                if n_matches >= min_matches:
                    group.create_dataset(key2, data=np.concatenate([mkpts1, mkpts2], axis=1).astype(np.float32))
                    cnt_pairs+=1
    gc.collect()
    t=time() -t
    print(f'Features matched in  {t:.4f} sec')
    return t

# Keypoints: LoFTR

In [ ]:
class LoFTRDataset(Dataset):
    def __init__(self, fnames1, fnames2, idxs1, idxs2, resize_small_edge_to, device):
        self.fnames1 = fnames1
        self.fnames2 = fnames2
        self.keys1 = [ fname.split('/')[-1] for fname in fnames1 ]
        self.keys2 = [ fname.split('/')[-1] for fname in fnames2 ]
        self.idxs1 = idxs1
        self.idxs2 = idxs2
        self.resize_small_edge_to = resize_small_edge_to
        self.device = device
        self.round_unit = 16

    def __len__(self):
        return len(self.images1)

    def load_torch_image(self, fname, device):
        img = cv2.imread(fname)
        original_shape = img.shape
        ratio = self.resize_small_edge_to / min([img.shape[0], img.shape[1]])
        w = int(img.shape[1] * ratio) # int( (img.shape[1] * ratio) // self.round_unit * self.round_unit )
        h = int(img.shape[0] * ratio) # int( (img.shape[0] * ratio) // self.round_unit * self.round_unit )
        img_resized = cv2.resize(img, (w, h))
        img_resized = K.image_to_tensor(img_resized, False).float() /255.
        img_resized = K.color.bgr_to_rgb(img_resized)
        img_resized = K.color.rgb_to_grayscale(img_resized)
        return img_resized.to(device), original_shape

    def __getitem__(self, idx):
        fname1 = self.fnames1[idx]
        fname2 = self.fnames2[idx]
        image1, ori_shape_1 = self.load_torch_image(fname1, device)
        image2, ori_shape_2 = self.load_torch_image(fname2, device)

        return image1, image2, self.keys1[idx], self.keys2[idx], self.idxs1[idx], self.idxs2[idx], ori_shape_1, ori_shape_2

def get_loftr_dataloader(images1, images2, idxs1, idxs2, resize_small_edge_to, device, batch_size=1):
    dataset = LoFTRDataset(images1, images2, idxs1, idxs2, resize_small_edge_to, device)
    dataloader = DataLoader(
        dataset=dataset,
        shuffle=False,
        batch_size=batch_size,
        pin_memory=True,
        num_workers=2,
        drop_last=False
    )
    return dataset

def detect_loftr(img_fnames, index_pairs, feature_dir, device, file_keypoints, resize_small_edge_to=750, min_matches=15):
    t=time()

    matcher = LoFTR(pretrained=None)
    matcher.load_state_dict(torch.load("../input/loftr/pytorch/outdoor/1/loftr_outdoor.ckpt")['state_dict'])
    matcher = matcher.to(device).eval()

    fnames1, fnames2, idxs1, idxs2 = [], [], [], []
    for pair_idx in progress_bar(index_pairs):
        idx1, idx2 = pair_idx
        fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
        fnames1.append(fname1)
        fnames2.append(fname2)
        idxs1.append(idx1)
        idxs2.append(idx2)


    dataloader = get_loftr_dataloader( fnames1, fnames2, idxs1, idxs2, resize_small_edge_to, device)

    cnt_pairs = 0

    with h5py.File(file_keypoints, mode='w') as f_match:
        store_mkpts = {}
        for X in tqdm(dataloader):
            image1, image2, key1, key2, idx1, idx2, ori_shape_1, ori_shape_2 = X
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]

            with torch.no_grad():
                correspondences = matcher( {"image0": image1.to(device),"image1": image2.to(device)} )
                mkpts1 = correspondences['keypoints0'].cpu().numpy()
                mkpts2 = correspondences['keypoints1'].cpu().numpy()
                mconf  = correspondences['confidence'].cpu().numpy()

            mkpts1[:,0] *= (float(ori_shape_1[1]) / float(image1.shape[3]))
            mkpts1[:,1] *= (float(ori_shape_1[0]) / float(image1.shape[2]))

            mkpts2[:,0] *= (float(ori_shape_2[1]) / float(image2.shape[3]))
            mkpts2[:,1] *= (float(ori_shape_2[0]) / float(image2.shape[2]))

            n_matches = mconf.shape[0]

            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                group.create_dataset(key2, data=np.concatenate([mkpts1, mkpts2], axis=1).astype(np.float32))
                cnt_pairs+=1
                print (f'{key1}-{key2}: {n_matches} matches @ {cnt_pairs}th pair(loftr)')
            else:
                print (f'{key1}-{key2}: {n_matches} matches --> skipped')
    gc.collect()
    t=time() -t
    print(f'Features matched in  {t:.4f} sec')
    return t

# Keypoints: DKM

In [ ]:
class DKMDataset(Dataset):
    def __init__(self, fnames1, fnames2, resize_to, device):
        self.fnames1 = fnames1
        self.fnames2 = fnames2
        self.resize_to = resize_to
        self.device = device
        self.test_transform = get_tuple_transform_ops(
            resize=self.resize_to, normalize=True
        )


    def __len__(self):
        return len(self.fnames1)

    def __getitem__(self, idx):
        fname1 = self.fnames1[idx]
        fname2 = self.fnames2[idx]

        im1, im2 = Image.open(fname1), Image.open(fname2)
        ori_shape_1 = im1.size
        ori_shape_2 = im2.size
        image1, image2 = self.test_transform((im1, im2))
        return image1, image2, torch.tensor([idx]), torch.tensor(ori_shape_1), torch.tensor(ori_shape_2)

def get_dkm_dataloader(images1, images2, resize_to, device, batch_size=4):
    dataset = DKMDataset(images1, images2, resize_to, device)
    dataloader = DataLoader(
        dataset=dataset,
        shuffle=False,
        batch_size=batch_size,
        pin_memory=True,
        num_workers=2,
        drop_last=False
    )
    return dataloader

def get_dkm_mkpts(dkm_model, bimgs1, bimgs2, shapes1, shapes2, detection_threshold=0.5, num_features = 2000, min_matches=15):
    dense_matches, dense_certainty = dkm_model.match(bimgs1, bimgs2, batched=True)

    store_mkpts1, store_mkpts2, store_mconf = [], [], []
    # drop low confidence pairs
    for b in range(dense_matches.shape[0]):
        u_dense_matches = dense_matches[b, dense_certainty[b,...].sqrt() >= detection_threshold, :]
        u_dense_certainty = dense_certainty[b, dense_certainty[b,...].sqrt() >= detection_threshold]

        if u_dense_matches.shape[0] > num_features:
            u_dense_matches, u_dense_certainty = dkm_model.sample( u_dense_matches, u_dense_certainty, num=num_features)

        u_dense_matches = u_dense_matches.reshape((-1, 4))
        u_dense_certainty = u_dense_certainty.reshape((-1,))

        mkpts1 = u_dense_matches[:, :2]
        mkpts2 = u_dense_matches[:, 2:]

        w1, h1 = shapes1[b, :]
        w2, h2 = shapes2[b, :]

        mkpts1[:, 0] = ((mkpts1[:, 0] + 1)/2) * w1
        mkpts1[:, 1] = ((mkpts1[:, 1] + 1)/2) * h1

        mkpts2[:, 0] = ((mkpts2[:, 0] + 1)/2) * w2
        mkpts2[:, 1] = ((mkpts2[:, 1] + 1)/2) * h2

        mkpts1 = mkpts1.cpu().detach().numpy()
        mkpts2 = mkpts2.cpu().detach().numpy()
        mconf  = u_dense_certainty.sqrt().cpu().detach().numpy()

        if mconf.shape[0] > min_matches:
            try:
                # calc Fundamental matrix from keypoints
                F, inliers = cv2.findFundamentalMat(mkpts1, mkpts2, cv2.USAC_MAGSAC, 0.200, 0.999, 2000)
                inliers = inliers > 0
                mkpts1 = mkpts1[inliers[:,0]]
                mkpts2 = mkpts2[inliers[:,0]]
                mconf  = mconf[inliers[:,0]]
            except:
                pass
        store_mkpts1.append(mkpts1)
        store_mkpts2.append(mkpts2)
        store_mconf.append(mconf)
    return store_mkpts1, store_mkpts2, store_mconf

def detect_dkm(
    img_fnames, index_pairs, feature_dir, device, file_keypoints,
    resize_to=(540, 720),
    detection_threshold=0.4,
    num_features=2000,
    min_matches=15
):
    t=time()
    dkm_model = DKMv3_outdoor(device=device)
    dkm_model.upsample_preds=False

    fnames1, fnames2 = [], []
    for pair_idx in progress_bar(index_pairs):
        idx1, idx2 = pair_idx
        fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
        fnames1.append(fname1)
        fnames2.append(fname2)

    cnt_pairs = 0
    with h5py.File(file_keypoints, mode='w') as f_match:
        dataloader = get_dkm_dataloader(fnames1, fnames2, resize_to, device, batch_size=4)
        for X in tqdm(dataloader):
            images1, images2, idxs, shapes1, shapes2 = X
            store_mkpts1, store_mkpts2, store_mconf = get_dkm_mkpts(
                dkm_model, images1.to(device), images2.to(device), shapes1, shapes2,
                detection_threshold=detection_threshold, num_features = num_features, min_matches=min_matches,
            )

            for b in range(images1.shape[0]):
                mkpts1 = store_mkpts1[b]
                mkpts2 = store_mkpts2[b]
                mconf = store_mconf[b]
                file1 = fnames1[idxs[b]]
                file2 = fnames2[idxs[b]]
                key1, key2 = file1.split('/')[-1], file2.split('/')[-1]

                n_matches = mconf.shape[0]

                group  = f_match.require_group(key1)
                if n_matches >= min_matches:
                    group.create_dataset(key2, data=np.concatenate([mkpts1, mkpts2], axis=1).astype(np.float32))
                    cnt_pairs+=1
                    print (f'{key1}-{key2}: {n_matches} matches @ {cnt_pairs}th pair(dkm)')
                else:
                    print (f'{key1}-{key2}: {n_matches} matches --> skipped')

    gc.collect()
    t=time() -t
    print(f'Features matched in  {t:.4f} sec')
    return t

# Keypoints merger

In [ ]:
def get_unique_idxs(A, dim=0):
    # https://stackoverflow.com/questions/72001505/how-to-get-unique-elements-and-their-firstly-appeared-indices-of-a-pytorch-tenso
    unique, idx, counts = torch.unique(A, dim=dim, sorted=True, return_inverse=True, return_counts=True)
    _, ind_sorted = torch.sort(idx, stable=True)
    cum_sum = counts.cumsum(0)
    cum_sum = torch.cat((torch.tensor([0],device=cum_sum.device), cum_sum[:-1]))
    first_indices = ind_sorted[cum_sum]
    return first_indices

def get_keypoint_from_h5(fp, key1, key2):
    rc = -1
    try:
        kpts = np.array(fp[key1][key2])
        rc = 0
        return (rc, kpts)
    except:
        return (rc, None)

def get_keypoint_from_multi_h5(fps, key1, key2):
    list_mkpts = []
    for fp in fps:
        rc, mkpts = get_keypoint_from_h5(fp, key1, key2)
        if rc == 0:
            list_mkpts.append(mkpts)
    if len(list_mkpts) > 0:
        list_mkpts = np.concatenate(list_mkpts, axis=0)
    else:
        list_mkpts = None
    return list_mkpts

def keypoints_merger(
    img_fnames,
    index_pairs,
    files_keypoints,
    feature_dir = 'featureout',
):
    # open h5 files
    fps = [ h5py.File(file, mode="r") for file in files_keypoints ]

    # temprary file
    with h5py.File(f'{feature_dir}/merge_tmp.h5', mode='w') as f_match:
        counter = 0
        for pair_idx in progress_bar(index_pairs):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]

            # extract keypoints
            mkpts = get_keypoint_from_multi_h5(fps, key1, key2)
            if mkpts is None:
                print(f"skipped key1={key1}, key2={key2}")
                continue

            print (f'{key1}-{key2}: {mkpts.shape[0]} matches')
            # regist tmp file
            group  = f_match.require_group(key1)
            group.create_dataset(key2, data=mkpts)
            counter += 1
    print( f"Ensembled pairs : {counter} pairs" )
    for fp in fps:
        fp.close()

    # Let's find unique loftr pixels and group them together.
    kpts = defaultdict(list)
    match_indexes = defaultdict(dict)
    total_kpts=defaultdict(int)
    with h5py.File(f'{feature_dir}/merge_tmp.h5', mode='r') as f_match:
        for k1 in f_match.keys():
            group  = f_match[k1]
            for k2 in group.keys():
                matches = group[k2][...]
                total_kpts[k1]
                kpts[k1].append(matches[:, :2])
                kpts[k2].append(matches[:, 2:])
                current_match = torch.arange(len(matches)).reshape(-1, 1).repeat(1, 2)
                current_match[:, 0]+=total_kpts[k1]
                current_match[:, 1]+=total_kpts[k2]
                total_kpts[k1]+=len(matches)
                total_kpts[k2]+=len(matches)
                match_indexes[k1][k2]=current_match

    for k in kpts.keys():
        kpts[k] = np.round(np.concatenate(kpts[k], axis=0))
    unique_kpts = {}
    unique_match_idxs = {}
    out_match = defaultdict(dict)
    for k in kpts.keys():
        uniq_kps, uniq_reverse_idxs = torch.unique(torch.from_numpy(kpts[k]),dim=0, return_inverse=True)
        unique_match_idxs[k] = uniq_reverse_idxs
        unique_kpts[k] = uniq_kps.numpy()
    for k1, group in match_indexes.items():
        for k2, m in group.items():
            m2 = deepcopy(m)
            m2[:,0] = unique_match_idxs[k1][m2[:,0]]
            m2[:,1] = unique_match_idxs[k2][m2[:,1]]
            mkpts = np.concatenate([unique_kpts[k1][ m2[:,0]],
                                    unique_kpts[k2][  m2[:,1]],
                                   ],
                                   axis=1)
            unique_idxs_current = get_unique_idxs(torch.from_numpy(mkpts), dim=0)
            m2_semiclean = m2[unique_idxs_current]
            unique_idxs_current1 = get_unique_idxs(m2_semiclean[:, 0], dim=0)
            m2_semiclean = m2_semiclean[unique_idxs_current1]
            unique_idxs_current2 = get_unique_idxs(m2_semiclean[:, 1], dim=0)
            m2_semiclean2 = m2_semiclean[unique_idxs_current2]
            out_match[k1][k2] = m2_semiclean2.numpy()
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp:
        for k, kpts1 in unique_kpts.items():
            f_kp[k] = kpts1

    with h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:
        for k1, gr in out_match.items():
            group  = f_match.require_group(k1)
            for k2, match in gr.items():
                group[k2] = match
    return

# Submission utilities

In [ ]:
def arr_to_str(a):
    return ';'.join([str(x) for x in a.reshape(-1)])

# Function to create a submission file.
def create_submission(out_results, data_dict):
    with open(f'submission.csv', 'w') as f:
        f.write('image_path,dataset,scene,rotation_matrix,translation_vector\n')
        for dataset in data_dict:
            if dataset in out_results:
                res = out_results[dataset]
            else:
                res = {}
            for scene in data_dict[dataset]:
                if scene in res:
                    scene_res = res[scene]
                else:
                    scene_res = {"R":{}, "t":{}}
                for image in data_dict[dataset][scene]:
                    if image in scene_res:
                        print (image)
                        R = scene_res[image]['R'].reshape(-1)
                        T = scene_res[image]['t'].reshape(-1)
                    else:
                        R = np.eye(3).reshape(-1)
                        T = np.zeros((3))
                    f.write(f'{image},{dataset},{scene},{arr_to_str(R)},{arr_to_str(T)}\n')

# Main

In [ ]:
src = '/kaggle/input/image-matching-challenge-2024'
DEBUG = False
DUMP = False

# Get data from csv.
data_dict = {}
with open(f'{src}/sample_submission.csv', 'r') as f:
    for i, l in enumerate(f):
        # Skip header.
        if l and i > 0:
            image, dataset, scene, _, _ = l.strip().split(',')
            if dataset not in data_dict:
                data_dict[dataset] = {}
            if scene not in data_dict[dataset]:
                data_dict[dataset][scene] = []
            data_dict[dataset][scene].append(image)

            #if len(data_dict[dataset][scene]) == 21:
            #    break

In [ ]:
for dataset in data_dict:
    for scene in data_dict[dataset]:
        print(f'{dataset} / {scene} -> {len(data_dict[dataset][scene])} images')

church / church -> 41 images


In [ ]:
out_results = {}
timings = {"shortlisting":[],
           "feature_detection": [],
           "feature_matching":[],
           "RANSAC": [],
           "Reconstruction": []}

In [ ]:
gc.collect()
datasets = []
for dataset in data_dict:
    datasets.append(dataset)

for dataset in datasets:
    print(dataset)
    if dataset not in out_results:
        out_results[dataset] = {}
    for scene in data_dict[dataset]:
        print(scene)
        # Fail gently if the notebook has not been submitted and the test data is not populated.
        # You may want to run this on the training data in that case?
        img_dir = f'{src}/test/{dataset}/images'
        if not os.path.exists(img_dir):
            continue
        # Wrap the meaty part in a try-except block.
        try:
            out_results[dataset][scene] = {}
            img_fnames = [f'{src}/{x}' for x in data_dict[dataset][scene]]
            print (f"Got {len(img_fnames)} images")
            feature_dir = f'featureout/{dataset}_{scene}'
            if not os.path.isdir(feature_dir):
                os.makedirs(feature_dir, exist_ok=True)

            #############################################################
            # get image pairs
            #############################################################
            t=time()
            index_pairs = get_image_pairs_shortlist(img_fnames,
                                  sim_th = 1.0, # should be strict
                                  min_pairs = 50, # we select at least min_pairs PER IMAGE with biggest similarity
                                  exhaustive_if_less = 50,
                                  device=device)
            t=time() -t
            timings['shortlisting'].append(t)
            print (f'{len(index_pairs)}, pairs to match, {t:.4f} sec')
            gc.collect()

            #############################################################
            # get keypoints
            #############################################################
            files_keypoints = []
            if CONFIG.use_superglue:
                resize_to = CONFIG.params_sg["resize_to"]
                file_keypoints = f"{feature_dir}/matches_superglue_{resize_to}pix.h5"
                !rm -rf {file_keypoints}
                t = detect_superglue(
                    img_fnames, index_pairs, feature_dir, device,
                    CONFIG.params_sg["sg_config"], file_keypoints,
                    resize_to=CONFIG.params_sg["resize_to"],
                    min_matches=CONFIG.params_sg["min_matches"],
                )
                gc.collect()
                files_keypoints.append( file_keypoints )
                timings['feature_matching'].append(t)

            if CONFIG.use_aliked_lightglue:
                model_name = "aliked"
                file_keypoints = f'{feature_dir}/matches_lightglue_{model_name}.h5'
                t = detect_lightglue_common(
                    img_fnames, model_name, index_pairs, feature_dir, device, file_keypoints,
                    resize_to=CONFIG.params_aliked_lightglue["resize_to"],
                    detection_threshold=CONFIG.params_aliked_lightglue["detection_threshold"],
                    num_features=CONFIG.params_aliked_lightglue["num_features"],
                    min_matches=CONFIG.params_aliked_lightglue["min_matches"],
                )
                gc.collect()
                files_keypoints.append(file_keypoints)
                timings['feature_matching'].append(t)

            if CONFIG.use_doghardnet_lightglue:
                model_name = "doghardnet"
                file_keypoints = f'{feature_dir}/matches_lightglue_{model_name}.h5'
                t = detect_lightglue_common(
                    img_fnames, model_name, index_pairs, feature_dir, device, file_keypoints,
                    resize_to=CONFIG.params_doghardnet_lightglue["resize_to"],
                    detection_threshold=CONFIG.params_doghardnet_lightglue["detection_threshold"],
                    num_features=CONFIG.params_doghardnet_lightglue["num_features"],
                    min_matches=CONFIG.params_doghardnet_lightglue["min_matches"],
                )
                gc.collect()
                files_keypoints.append(file_keypoints)
                timings['feature_matching'].append(t)

            if CONFIG.use_superpoint_lightglue:
                model_name = "superpoint"
                file_keypoints = f'{feature_dir}/matches_lightglue_{model_name}.h5'
                t = detect_lightglue_common(
                    img_fnames, model_name, index_pairs, feature_dir, device, file_keypoints,
                    resize_to=CONFIG.params_superpoint_lightglue["resize_to"],
                    detection_threshold=CONFIG.params_superpoint_lightglue["detection_threshold"],
                    num_features=CONFIG.params_superpoint_lightglue["num_features"],
                    min_matches=CONFIG.params_superpoint_lightglue["min_matches"],
                )
                gc.collect()
                files_keypoints.append(file_keypoints)
                timings['feature_matching'].append(t)

            if CONFIG.use_loftr:
                file_keypoints = f'{feature_dir}/matches_loftr_{CONFIG.params_loftr["resize_small_edge_to"]}pix.h5'
                t = detect_loftr(
                    img_fnames, index_pairs, feature_dir, device, file_keypoints,
                    resize_small_edge_to=CONFIG.params_loftr["resize_small_edge_to"],
                    min_matches=CONFIG.params_loftr["min_matches"],
                )
                gc.collect()
                files_keypoints.append( file_keypoints )
                timings['feature_matching'].append(t)

            if CONFIG.use_dkm:
                file_keypoints = f'{feature_dir}/matches_dkm.h5'
                t = detect_dkm(
                    img_fnames, index_pairs, feature_dir, device, file_keypoints,
                    resize_to=CONFIG.params_dkm["resize_to"],
                    detection_threshold=CONFIG.params_dkm["detection_threshold"],
                    num_features=CONFIG.params_dkm["num_features"],
                    min_matches=CONFIG.params_dkm["min_matches"]
                )
                gc.collect()
                files_keypoints.append(file_keypoints)
                timings['feature_matching'].append(t)

            #############################################################
            # merge keypoints
            #############################################################
            keypoints_merger(
                img_fnames,
                index_pairs,
                files_keypoints,
                feature_dir = feature_dir,
            )

            #############################################################
            # regist keypoints from h5 into colmap db
            #############################################################
            database_path = f'{feature_dir}/colmap.db'
            if os.path.isfile(database_path):
                os.remove(database_path)
            gc.collect()
            import_into_colmap(img_dir, feature_dir=feature_dir,database_path=database_path)
            output_path = f'{feature_dir}/colmap_rec'

            #############################################################
            # Calculate fundamental matrix with colmap api
            #############################################################
            t=time()
            options = pycolmap.SiftMatchingOptions()
            options.confidence = 0.9999
            options.max_num_trials = 20000
            pycolmap.match_exhaustive(database_path, sift_options=options)
            t=time() - t
            timings['RANSAC'].append(t)
            print(f'RANSAC in  {t:.4f} sec')

            #############################################################
            # Execute bundle adjustmnet with colmap api
            # --> Bundle adjustment Calcs Camera matrix, R and t
            #############################################################
            t=time()
            # By default colmap does not generate a reconstruction if less than 10 images are registered. Lower it to 3.
            mapper_options = pycolmap.IncrementalMapperOptions()
            mapper_options.min_model_size = 3
            os.makedirs(output_path, exist_ok=True)
            maps = pycolmap.incremental_mapping(database_path=database_path, image_path=img_dir, output_path=output_path, options=mapper_options)
            print(maps)
            clear_output(wait=False)
            t=time() - t
            timings['Reconstruction'].append(t)
            print(f'Reconstruction done in  {t:.4f} sec')

            #############################################################
            # Extract R,t from maps
            #############################################################
            imgs_registered  = 0
            best_idx = None
            list_num_images = []
            print ("Looking for the best reconstruction")
            if isinstance(maps, dict):
                for idx1, rec in maps.items():
                    print (idx1, rec.summary())
                    list_num_images.append( len(rec.images) )
                    if len(rec.images) > imgs_registered:
                        imgs_registered = len(rec.images)
                        best_idx = idx1
            list_num_images = np.array(list_num_images)
            print(f"list_num_images = {list_num_images}")
            if best_idx is not None:
                print (maps[best_idx].summary())
                for k, im in maps[best_idx].images.items():
                    key1 = f'test/{dataset}/images/{im.name}'
                    out_results[dataset][scene][key1] = {}
                    out_results[dataset][scene][key1]["R"] = deepcopy(im.rotmat())
                    out_results[dataset][scene][key1]["t"] = deepcopy(np.array(im.tvec))

            print(f'Registered: {dataset} / {scene} -> {len(out_results[dataset][scene])} images')
            print(f'Total: {dataset} / {scene} -> {len(data_dict[dataset][scene])} images')
            create_submission(out_results, data_dict)
            gc.collect()
        except:
            pass

church
church
Got 41 images


100%|██████████| 41/41 [00:03<00:00, 10.32it/s]


820, pairs to match, 14.1369 sec


  7%|▋         | 3/41 [00:00<00:06,  5.99it/s]

aliked > kpts.shape=(8192, 2), descs.shape=(8192, 128)
aliked > kpts.shape=(8192, 2), descs.shape=(8192, 128)
aliked > kpts.shape=(8192, 2), descs.shape=(8192, 128)
aliked > kpts.shape=(8192, 2), descs.shape=(8192, 128)


...

 23%|██▎       | 187/820 [02:12<07:28,  1.41it/s]

00083.png-00074.png: 821 matches @ 183th pair(aliked+lightglue)


 23%|██▎       | 188/820 [02:13<07:27,  1.41it/s]

00013.png-00102.png: 179 matches @ 184th pair(aliked+lightglue)


 23%|██▎       | 189/820 [02:14<07:27,  1.41it/s]

# Result

In [ ]:
!cat submission.csv